In [1]:
!pip install datasets
!pip install accelerate -U
!pip install evaluate
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

### give your file path
dataset_path = "train_v2_drcat_02.csv"
df = pd.read_csv(dataset_path, header = 0)

### get the labels
label = df['label']
label = label.astype("int")
# df.drop(columns='label', axis = 1, inplace = True)
df.drop(columns='RDizzl3_seven', axis = 1, inplace=True)
df.drop(columns='prompt_name', axis = 1, inplace=True)
df.drop(columns='source', axis = 1, inplace=True)
df.dropna(subset=['text'], inplace=True)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [9]:
!pip install transformers
!pip install protobuf
from transformers import AutoTokenizer, DebertaV2Tokenizer, DebertaV2ForSequenceClassification
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

### testing
# model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-small", num_labels = 2)
# tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-small")
model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-xsmall", num_labels = 2)
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize_func(dataset):
    return tokenizer(list(dataset['text']), padding = "max_length", truncation = True)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_tokenized_datasets = train_dataset.map(tokenize_func, batched = True)
test_tokenized_datasets = test_dataset.map(tokenize_func, batched = True)
train_tokenized_datasets.set_format(type='torch', columns = ['input_ids', 'attention_mask', 'label'])
test_tokenized_datasets.set_format(type='torch', columns = ['input_ids', 'attention_mask', 'label'])

# Drop columns from the tokenized datasets
columns_to_drop = ["text", "__index_level_0__", "token_type_ids"]  # Specify columns to drop
train_tokenized_datasets = train_tokenized_datasets.remove_columns(columns_to_drop)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(columns_to_drop)

print(train_tokenized_datasets)
print(test_tokenized_datasets)

Map:   0%|          | 0/35894 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/8974 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 35894
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 8974
})


In [11]:
from transformers import Trainer, TrainingArguments
import evaluate
import torch.nn.functional as F

batch_size = 2
args = TrainingArguments(output_dir = "LLMvHuman_finetune",
                               num_train_epochs = 2,
                               learning_rate = 2e-5,
                               per_device_train_batch_size = batch_size,
                               per_device_eval_batch_size = batch_size,
                               weight_decay = 0.01,
                               evaluation_strategy = "epoch",
                               disable_tqdm = False,
                               logging_steps = 10000,
                               push_to_hub = False,
                              #  log_level = "error",
                               save_strategy="epoch",
                               load_best_model_at_end=True
                              #  fp16=True,
                              #  fp16_backend="auto"
                              )
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
torch.cuda.empty_cache()

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=args,                  # training arguments, defined above
    train_dataset=train_tokenized_datasets,         # training dataset
    eval_dataset=test_tokenized_datasets,             # evaluation dataset
    tokenizer = tokenizer,
    # use_amp=True,
    compute_metrics = compute_metrics
)

# Step 8: Train the model
trainer.train()

# Step 9: Evaluate the model
trainer.evaluate()

/home/mohbe.r/.local/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.056000,0.023763,0.997103
2,0.011600,0.029310,0.996323


{'eval_loss': 0.02376273274421692,
 'eval_accuracy': 0.9971027412525072,
 'eval_runtime': 99.2751,
 'eval_samples_per_second': 90.395,
 'eval_steps_per_second': 45.198,
 'epoch': 2.0}

In [13]:
model.save_pretrained("./model", from_pt=True) 

In [15]:
tokenizer.save_pretrained("./model" )

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/spm.model',
 './model/added_tokens.json',
 './model/tokenizer.json')

In [16]:
trainer.save_model("./my_model")

In [27]:
from transformers import AutoTokenizer, DebertaV2Tokenizer, DebertaV2ForSequenceClassification
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

# ### testing
# # model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-small", num_labels = 2)
# # tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-small")
# model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-xsmall", num_labels = 2)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")



# Load the fine-tuned model
model_path = "./my_model"  # Specify the path to your fine-tuned model
model = DebertaV2ForSequenceClassification.from_pretrained(model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Example text for prediction
text = "Essay written by human. Will my model classify this correctly? I am a bit skeptical. See the word embeddings are positional encodings. Idk what else the model will learn apart from context w.r.t the content. Have to see!"
# text = "fuck yes.....this is human"
# text = "In the era of advancing technology, the emergence of Large Language Models (LLMs) like GPT has stirred debates on the potential rivalry between artificial and human intelligence.  LLMs, powered by vast datasets and complex algorithms, exhibit remarkable linguistic abilities. However, they lack the nuanced understanding, consciousness, and emotional depth inherent in human intelligence.  Despite disparities, LLMs and humans can collaborate effectively. LLMs excel in data processing, while humans contribute contextual understanding and ethical judgment. Ethical concerns, including privacy and algorithmic bias, necessitate careful regulation and oversight. Furthermore, societal implications such as job displacement and inequality must be addressed.  In conclusion, while LLMs offer significant potential, their integration should be approached with caution. Collaborative efforts can ensure that AI enhances, rather than undermines, human well-being and autonomy."""
# Tokenize input text
inputs = tokenizer(text, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted class probabilities
probs = torch.softmax(outputs.logits, dim=-1)

# Get predicted class label
predicted_class = torch.argmax(probs, dim=-1).item()

print("Predicted class label:", predicted_class) #it needs to print 0
print("Predicted class probabilities:", probs)

Predicted class label: 0
Predicted class probabilities: tensor([[9.9999e-01, 1.4389e-05]])
